In [1]:
# temporary solution to crashing
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#import torchvision.transforms as transforms
from torch import Tensor
from torchvision.ops import Conv2dNormActivation

import matplotlib.pyplot as plt
import numpy as np
#import mysql.connector as connector

from pathlib import Path

In [3]:
# issues to debug: since groups == hidden_dim and hidden_dim == int(round(in_channels * expand_ratio))
# the in_channels are not divisible by groups

class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio, norm_layer=None):
        super().__init__()

        assert stride == 1 or stride == 2, 'stride must be 1 or 2'

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        hidden_dim = int(round(in_channels * expand_ratio))
        # print(hidden_dim)

        layers = [] 

        if expand_ratio != 1:
            # add expansion 
            layers.append(Conv2dNormActivation(in_channels, hidden_dim, kernel_size=3, stride=stride, 
                                               norm_layer=norm_layer, activation_layer=nn.ReLU6))
        # depthwise conv => pointwise => norm_layer
        layers.extend([Conv2dNormActivation(hidden_dim, hidden_dim, kernel_size=3, stride=stride, groups=hidden_dim, norm_layer=norm_layer, activation_layer=nn.ReLU6), 
                                                nn.Conv2d(hidden_dim, out_channels, 1, 1, 0, bias=False), 
                                                norm_layer(out_channels)])
        
        self.conv = nn.Sequential(*layers)
        
        self.stride = stride
        self.out_channels = out_channels
        self.in_channels = in_channels
        self._is_cn = stride > 1 # downsample indicator - what does this mean?
        self.use_res_conn = stride == 1 and in_channels == out_channels

    def forward(self, x):
        if self.use_res_conn:
            return x + self.conv(x)
        else:                
            return self.conv(x)
        

In [4]:
# testing the inverted residual layers
# everything seems to work except there is an issue with "groups"

test_inv = InvertedResidual(3, 30, 1, 2)
test_input = torch.randn(1, 3, 32, 32)

test_inv(test_input).shape

torch.Size([1, 30, 32, 32])

In [5]:
def _make_divisible(v, divisor, min_value=None): # rounds to nearest multiple of divisor
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [ ]:
''' 
[x]: self.features produces a runtime error when batch_size = 1, [which means there MIGHT be something erroneous with the implementation] 
-- the reason is because batchnorm expects more than 1 sample in a batch, so the implementation is fine
'''


class MobileNetV2(nn.Module):
    """
        MobileNet V2 main class

        Args:
            width_mult (float): Width multiplier - adjusts number of channels in each layer by this amount
            inverted_residual_setting: Network structure
            round_nearest (int): Round the number of channels in each layer to be a multiple of this number
            Set to 1 to turn off rounding
            block: Module specifying inverted residual building block for mobilenet
            norm_layer: Module specifying the normalization layer to use

        """
    def __init__(self, num_classes=1000, width_mult=1.0, inverted_residual_setting = None, round_nearest=8,
                  block = None, norm_layer = None, dropout = 0.2):
        super().__init__()
        if block is None:
            block = InvertedResidual

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        input_channel = 32 # adjustable
        last_channel = 1280 # adjustable

        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t - expansion factor, c - # of output after block, n - # of repitions, s - stride
                [1, 16, 1, 1],
                [6, 24, 2, 2],
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # build the first layer
        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        print('input_channel', input_channel)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)

        
        features = [
            Conv2dNormActivation(3, input_channel, kernel_size=3, stride=2, norm_layer=norm_layer, activation_layer=nn.ReLU6)
        ]

        # build the inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c*width_mult, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t, norm_layer=norm_layer))
                input_channel = output_channel 
                
        # build last severals
        features.append(Conv2dNormActivation(input_channel, self.last_channel, kernel_size=1, norm_layer=norm_layer, activation_layer=nn.ReLU6))

        # complete CNN architecture
        self.features = nn.Sequential(*features) 

        # weight initialization
        for m in self.modules():
            # print('initializing weights')
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)
    
    def _forward_impl(self, x):
        x = self.features(x) # pass input through layers
        
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1)) # takes the average value for each each channel
        x = torch.flatten(x, 1) # flatten for linear layer
        # x = self.classifier(x) # classify
        return x

    def forward(self, x):
        return self._forward_impl(x)


In [14]:
test_mod = MobileNetV2(2)

test_in = torch.randn(1, 3, 32, 32)

input_channel 32


In [15]:
test_mod(test_in).shape

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 144, 1, 1])